In [ ]:
import pprint
from typing import List
from matrix import Matrix

# Матрица

In [ ]:
a = Matrix([[1, 2], [3, 4], [5, 6]])
b = Matrix([[0, 2], [2, 0], [2, 0]])
print( b.transpose() * a , a)

c = Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]])

print(c.determinant()) # Должен быть ноль

d = Matrix([
    [2, 3, 4],
    [5, 6, 7],
    [8, 9, 10]
])
print(c.determinant())

print(d[1])



[16, 20]
[2, 4]
 
[1, 2]
[3, 4]
[5, 6]

0.0
0.0
[5, 6, 7]


# PCA

## Easy

### Метод гауса

In [ ]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    n = A.size[0]
    augmented = A.augment(b)  # Расширенная матрица [A|b]
    
    # Прямой ход: приведение к ступенчатому виду
    rank = 0
    pivot_cols = []
    for col in range(augmented.cols - 1):  # Исключаем последний столбец (вектор b)
        # Поиск ненулевого элемента в текущем столбце
        pivot_row = -1
        for row in range(rank, n):
            if abs(augmented[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row == -1:
            continue  # Все элементы столбца нулевые
        
        # Перестановка строк
        augmented.swap_rows(rank, pivot_row)
        # Нормализация ведущей строки
        pivot = augmented[rank][col]
        for c in range(col, augmented.cols):
            augmented[rank][c] /= pivot
        # Исключение элементов ниже ведущего
        for row in range(rank + 1, n):
            factor = augmented[row][col]
            for c in range(col, augmented.cols):
                augmented[row][c] -= factor * augmented[rank][c]
        pivot_cols.append(col)
        rank += 1

    # Проверка на несовместность
    for row in range(rank, n):
        if abs(augmented[row][-1]) > 1e-10:
            raise ValueError("Система несовместна")

    # Определение свободных переменных
    all_vars = list(range(A.))
    leading_vars = pivot_cols
    free_vars = [var for var in all_vars if var not in leading_vars]
    
    # Если нет свободных переменных: единственное решение
    if not free_vars:
        x = [0] * A.cols
        for row in reversed(range(rank)):
            x[leading_vars[row]] = augmented[row][-1]
            for col in range(leading_vars[row] + 1, A.cols):
                x[leading_vars[row]] -= augmented[row][col] * x[col]
        return [Matrix([x])]

    # Построение базисных векторов
    basis = []
    for var in free_vars:
        solution = [0] * A.cols
        solution[var] = 1  # Свободная переменная = 1
        # Вычисление главных переменных
        for row in reversed(range(rank)):
            col = leading_vars[row]
            solution[col] = augmented[row][-1]
            for c in range(col + 1, A.cols):
                solution[col] -= augmented[row][c] * solution[c]
        basis.append(Matrix([solution]))
    
    return basis


In [ ]:

A = Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
b = Matrix([[6], [15], [24]])

solutions = gauss_solver(A, b)  # Возвращает базисные векторы (если система неопределённа)

AttributeError: 'Matrix' object has no attribute 'rows'